[View in Colaboratory](https://colab.research.google.com/github/somnathrakshit/Code/blob/master/keras_transfer.ipynb)

In [1]:
!ls
# !rm -rf dogscats/
# !kill -9 -1
# !pip install -U -q PyDrive kaggle keras

def download():
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # Download a file based on its file ID.
    #
    # A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
    file_id = '1uOHcurx42GonrO5xgwENMA3n2SfOZW2t'
    file_name = 'data.zip'
    print('Started Download')
    downloaded = drive.CreateFile({'id': file_id})
    downloaded.GetContentFile(file_name)
    print('Download Complete')

data  datalab  data.zip


In [0]:
# download()
# !unzip -q data.zip

In [0]:
img_height = 700
img_width = 460

In [0]:
from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

In [12]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.topology.InputLayer object at 0x7f84848f1a90> False
<keras.layers.convolutional.Conv2D object at 0x7f8484891978> False
<keras.layers.convolutional.Conv2D object at 0x7f847ecdcd30> False
<keras.layers.pooling.MaxPooling2D object at 0x7f84808a6c88> False
<keras.layers.convolutional.Conv2D object at 0x7f848091b240> False
<keras.layers.convolutional.Conv2D object at 0x7f847fcc17b8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f847fcd10f0> False
<keras.layers.convolutional.Conv2D object at 0x7f847fce7518> False
<keras.layers.convolutional.Conv2D object at 0x7f847fc95278> False
<keras.layers.convolutional.Conv2D object at 0x7f847fca90b8> False
<keras.layers.pooling.MaxPooling2D object at 0x7f847fc40080> False
<keras.layers.convolutional.Conv2D object at 0x7f847fc6a5c0> False
<keras.layers.convolutional.Conv2D object at 0x7f847fbffe48> False
<keras.layers.convolutional.Conv2D object at 0x7f847fbc04e0> False
<keras.layers.pooling.MaxPooling2D object at 0x7f847fc2b978> Fa

In [13]:
from keras.preprocessing.image import ImageDataGenerator

data_path = 'data'

# Change the batchsize according to your system RAM
train_batchsize = 8
val_batchsize = 8

datagen = ImageDataGenerator(
      validation_split=0.2,
#       rescale=1./255,
#       rotation_range=20,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=train_batchsize,
    subset='training',
    class_mode='categorical')
 

validation_generator = datagen.flow_from_directory(
    data_path,
    target_size=(img_height, img_width),
    batch_size=val_batchsize,
    class_mode='categorical',
    subset='validation',
    shuffle=False)

Found 6331 images belonging to 8 classes.
Found 1578 images belonging to 8 classes.


In [14]:
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 21, 14, 512)       14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 150528)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              154141696 
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 8200      
Total params: 168,864,584
Trainable params: 161,229,320
Non-trainable params: 7,635,264
_________________________________________________________________


In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=False),
              metrics=['accuracy'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples//train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples//validation_generator.batch_size,
      verbose=1
)
 
# Save the model
model.save('small_last4.h5')

Epoch 1/30
791/791 [==============================] - 956s 1s/step - loss: 1.8925 - acc: 0.4122 - val_loss: 1.7021 - val_acc: 0.4308
Epoch 2/30
119/791 [===>..........................] - ETA: 11:09 - loss: 1.5639 - acc: 0.4601

211/791 [=======>......................] - ETA: 9:36 - loss: 1.5723 - acc: 0.4631